<a href="https://colab.research.google.com/github/Art-phys/Lesson_HF_LR/blob/main/Lesson_HF_RL_Unit4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Блок 4: Создайте свой первый алгоритм обучения с глубоким подкреплением с помощью PyTorch: Reinforce. И проверьте его надежность 💪

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit6/thumbnail.png" alt="thumbnail" width="50%"/>

В этом блокноте вы с нуля создадите свой первый алгоритм обучения с глубоким подкреплением: Reinforcement (также называемый градиентом политики Монте-Карло).

Усиление (Reinforce) - это *метод, основанный на политике*: алгоритм глубокого обучения с подкреплением, который пытается **оптимизировать политику напрямую, не используя функцию значения действия**.

Точнее, Усиление (Reinforce) - это *Метод градиента политики*, подкласс *методов, основанных на политике*, который направлен **на оптимизацию политики непосредственно путем оценки весов оптимальной политики с использованием градиентного подъема**.

Чтобы проверить его надежность, мы собираемся обучить его в двух разных простых средах:
- Cartpole-v1 (Столб тележки-v1)
- PixelcopterEnv (Пиксельный вертолет)

⬇️ Вот пример того, чего **вы достигнете в конце этом блокноте.** ⬇️

  <img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit6/envs.gif" alt="Environments" width="50%"/>


### 🎮 Окружающая среда: 

- [CartPole-v1](https://www.gymlibrary.dev/environments/classic_control/cart_pole/)
- [PixelCopter](https://pygame-learning-environment.readthedocs.io/en/latest/user/games/pixelcopter.html)

### 📚 RL-Библиотека: 

- Python
- PyTorch


 [GitHub Repo](https://github.com/huggingface/deep-rl-class/issues).

## Цели этого блокнота 🏆
В конце вы будете:
- Уметь **с нуля закодировать алгоритм усиления с использованием PyTorch.**
- Иметь возможность **протестировать надежность вашего агента, используя простые среды.**
- Иметь возможность **отправить своего обученного агента в центр** с хорошим видеоповтором и оценочным баллом 🔥 .

## Эта тетрадь взята из курса обучения с глубоким подкреплением
<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/deep-rl-course-illustration.jpg" alt="Deep RL Course illustration" width="50%"/>

На этом бесплатном курсе вы:

- 📖 Изучите глубокое обучение с подкреплением в **теории и на практике**.
- 🧑 💻 Научитесь **использовать известные библиотеки Deep RL**, такие как Stable Baselines3, RL Baselines3 Zoo, CleanRL и Sample Factory 2.0.
- 🤖 Обучите **агентов в уникальных условиях**

## Предпосылки 🏗️
Прежде чем погрузиться в блокнот, вам нужно:

🔲 📚 [Изучите градиенты политики, прочитав Unit 4](https://huggingface.co/deep-rl-course/unit4/introduction)

# Давайте запрограммируем алгоритм усиления с нуля 🔥


Чтобы пройти этот практический подход для сертификации, вам необходимо отправить свои обученные модели в Центр.

- Получите результат >= 350 для `Cart pole-v1`.
- Получите результат >= 5 для `Pixel Copter`.

Чтобы узнать свой результат, перейдите в таблицу лидеров и найдите свою модель, **the result = mean_reward - std_reward**. **Если вы не видите свою модель в таблице лидеров, перейдите в нижнюю часть страницы таблицы лидеров и нажмите на кнопку обновить**.

Для получения дополнительной информации о процессе сертификации ознакомьтесь с этим разделом 👉 https://huggingface.co/deep-rl-course/en/unit0/introduction#certification-process

# Совет 💡
Лучше запустить этот colab в копии на вашем Google Диске, чтобы **в случае истечения времени ожидания** у вас все еще была сохраненная записная книжка на вашем Google Диске, и вам не нужно было заполнять все с нуля.

Для этого вы можете либо нажать `Ctrl + S` либо `File > Save a copy in Google Drive.`

## Запустить на GPU 💪
- Чтобы **ускорить обучение агента, мы будем использовать графический процессор GPU. Перейдите в `Runtime > Change Runtime type`

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/gpu-step1.jpg" alt="GPU Step 1" width="50%">

- `Hardware Accelerator > GPU`

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/gpu-step2.jpg" alt="GPU Step 2" width="50%">

## Создание виртуального дисплея #

Во время записи нам нужно будет сгенерировать видео-повтор. Для этого с помощью colab **нам нужен виртуальный экран, чтобы иметь возможность отображать среду** (и, таким образом, записывать кадры).

Следовательно, в следующей ячейке будут установлены библиотеки, а также создан и запущен виртуальный экран 🖥

In [ ]:
%%capture
!apt install python-opengl
!apt install ffmpeg
!apt install xvfb
!pip install pyvirtualdisplay
!pip install pyglet==1.5.1

In [ ]:
# Virtual display
from pyvirtualdisplay import Display

virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

## Установка зависимостей #
Первым шагом является установка зависимостей:

- `gym`
- `gym-games`: Дополнительные условия в тренажерном зале, созданные с помощью Pygame.
- `huggingface_hub`: 🤗 работает как центральное место, где любой желающий может делиться моделями и наборами данных и изучать их. В нем есть управление версиями, метрики, визуализации и другие функции, которые позволят вам легко сотрудничать с другими.

Вы можете увидеть здесь все доступные модели усилителей 👉 https://huggingface.co/models?other=reinforce

И вы можете найти все модели обучения с глубоким подкреплением здесь 👉 https://huggingface.co/models?pipeline_tag=reinforcement-learning

In [ ]:
!pip install -r https://raw.githubusercontent.com/huggingface/deep-rl-class/main/notebooks/unit4/requirements-unit4.txt

## Импортируйте пакеты 📦
В дополнение к импорту установленных библиотек, мы также импортируем:

- `imageio`: Библиотека, которая поможет нам сгенерировать повтор видео


In [ ]:
import numpy as np

from collections import deque

import matplotlib.pyplot as plt
%matplotlib inline

# PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical

# Gym
import gym
import gym_pygame

# Hugging Face Hub
from huggingface_hub import notebook_login # Войти в учетную запись Hugging Face, чтобы иметь возможность загружать модели в Хаб.
import imageio

## Проверьте, есть ли у нас графический процессор

Если у вас есть GPU, вы должны увидеть `device:cuda0`

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
print(device)

Теперь мы готовы реализовать наш алгоритм усиления 🔥

# Первый агент: Играем в Картпол-v1 🤖

## Создайте среду CartPole и поймите, как это работает
### [Окружающая среда 🎮 ](https://www.gymlibrary.dev/environments/classic_control/cart_pole/)


### Почему мы используем такую простую среду, как CartPole-v1?
Как описано в [Советах и рекомендациях по обучению с подкреплением](https://stable-baselines3.readthedocs.io/en/master/guide/rl_tips.html),когда вы внедряете свой агент с нуля, вам нужно **убедиться, что он работает правильно, и найти ошибки в простых средах, прежде чем углубляться**. Поскольку находить ошибки будет намного проще в простых средах.


> Попробуйте подать какой-нибудь “признак жизни” в отношении проблем с игрушками


> Проверьте реализацию, заставив ее запускаться во все более сложных средах (вы можете сравнить результаты с RL zoo). Обычно для этого шага необходимо выполнить оптимизацию по гиперпараметру.
___
### Среда CartPole-v1

> Шест прикреплен с помощью шарнира без привода к тележке, которая движется по рельсам без трения. Маятник устанавливается вертикально на тележку, и цель состоит в том, чтобы уравновесить шест, прикладывая силы в левом и правом направлениях к тележке.



Итак, мы начнем с CartPole-v1. Цель состоит в том, чтобы толкать тележку влево или вправо **так, чтобы шест оставался в равновесии.**

Эпизод заканчивается, если:
- Угол наклона шеста больше, чем ±12°
- Положение тележки больше, чем ±2.4
- Продолжительность эпизода больше, чем 500

Мы получаем вознаграждение 💰 в размере +1 каждый раз, когда полюс остается в равновесии.

In [ ]:
env_id = "CartPole-v1"
# Создание env
env = gym.make(env_id)

# Создание среды оценки
eval_env = gym.make(env_id)

# Получение пространства состояний и пространства действий
s_size = env.observation_space.shape[0]
a_size = env.action_space.n

In [ ]:
print("_____OBSERVATION SPACE_____ \n")
print("The State Space is: ", s_size)
print("Sample observation", env.observation_space.sample()) # Получение случайного наблюдения

In [ ]:
print("\n _____ACTION SPACE_____ \n")
print("The Action Space is: ", a_size)
print("Action Space Sample", env.action_space.sample()) # Получение случайного действия

## Давайте создадим Reinforce архитектуру
Эта реализация основана на двух реализациях:
- [Официальный пример обучения с подкреплением PyTorch](https://github.com/pytorch/examples/blob/main/reinforcement_learning/reinforce.py)
- [Udacity Reinforce](https://github.com/udacity/deep-reinforcement-learning/blob/master/reinforce/REINFORCE.ipynb)
- [Улучшение интеграции с помощью Chris1nexus](https://github.com/huggingface/deep-rl-class/pull/95)

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit6/reinforce.png" alt="Reinforce" width="50%"/>

Итак, мы хотим:
- Два полностью соединенных слоя (fc1 и fc2).
- Использование ReLU в качестве функции активации fc1
- Использование Softmax для вывода распределения вероятностей по действиям

In [ ]:
class Policy(nn.Module):
    def __init__(self, s_size, a_size, h_size):
        super(Policy, self).__init__()
        # Создайте два полностью соединенных (fc-fully connected ) слоя

    def forward(self, x):
        # Определите прямой проход
        # состояние переходит в fc1, затем мы применяем функцию активации ReLU
        # выходы fc1 передаются в fc2
        # Мы выводим softmax
    
    def act(self, state):
        """
        Учитывая состояние, определите действие
        """
        state = torch.from_numpy(state).float().unsqueeze(0).to(device)
        probs = self.forward(state).cpu()
        m = Categorical(probs)
        action = np.argmax(m)
        return action.item(), m.log_prob(action)

### Solution

In [ ]:
class Policy(nn.Module):
    def __init__(self, s_size, a_size, h_size):
        super(Policy, self).__init__()
        self.fc1 = nn.Linear(s_size, h_size)
        self.fc2 = nn.Linear(h_size, a_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.softmax(x, dim=1)
    
    def act(self, state):
        state = torch.from_numpy(state).float().unsqueeze(0).to(device)
        probs = self.forward(state).cpu()
        m = Categorical(probs)
        action = np.argmax(m)
        return action.item(), m.log_prob(action)

Я совершаю ошибку, можете ли вы догадаться, где?

- Чтобы выяснить это, давайте сделаем пас вперед:

In [ ]:
debug_policy = Policy(s_size, a_size, 64).to(device)
debug_policy.act(env.reset())

- Здесь мы видим, что ошибка гласит `ValueError: аргумент value для log_prob должен быть тензором`

- Это означает, что `action` в `m.log_prob(action)` должно быть, тензором **, но это не так.**

- Ты знаешь почему? Проверьте функцию act и попытайтесь понять, почему она не работает.

Совет 💡 : Что-то не так в этой реализации. Помните, что **мы хотим выбрать действие из распределения вероятностей по действиям**.


### Правильное решение

In [ ]:
class Policy(nn.Module):
    def __init__(self, s_size, a_size, h_size):
        super(Policy, self).__init__()
        self.fc1 = nn.Linear(s_size, h_size)
        self.fc2 = nn.Linear(h_size, a_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.softmax(x, dim=1)
    
    def act(self, state):
        state = torch.from_numpy(state).float().unsqueeze(0).to(device)
        probs = self.forward(state).cpu()
        m = Categorical(probs)
        action = m.sample()
        return action.item(), m.log_prob(action)

Используя CartPole, отлаживать было проще, поскольку **мы знаем, что ошибка возникает из-за нашей интеграции, а не из-за нашей простой среды**.